
Based Arshia Azfal's code: Loading train from the defined train sessions dt5 files. Adding checkpoints to the loading of the datasets as they can take a while.

In [ ]:
import numpy as np
import os
from tqdm import tqdm as tqdm

from pathlib import Path

import sys
sys.path.append('/Users/theomaetz/Downloads/MA3/INL/Setup/lighthouse-main')

from lighthouse.data_loader.load_chuv import load_absd
from lighthouse import metadata

In [ ]:
os.environ["DATA_DIR"] = "/Volumes"
X = []
Dir = []
Train_Dir = []
training_sessions = metadata.get_training_sessions()

2024-10-18 00:08:32.043 | WARNING  | lighthouse.metadata:get_training_sessions:125 - Only found 65 out of 66 training sessions


In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path

# Function to read checkpoint from file
def read_checkpoint(filename='checkpoint.txt'):
    try:
        with open(filename, 'r') as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return 0  # Start from the beginning if no checkpoint exists

# Function to write checkpoint to file
def write_checkpoint(session_index, filename='checkpoint.txt'):
    with open(filename, 'w') as f:
        f.write(str(session_index + 1))

In [ ]:

# Load previous checkpoints if available
try:
    X = list(np.load('Train_data.npy', allow_pickle=True))
    Dir = list(np.load('Ch_train.npy', allow_pickle=True))
    Train_Dir = list(np.load('dir_train.npy', allow_pickle=True))
    start_session = read_checkpoint()  # Resume from the last saved session
    print("Found File")
    print(f"Starting from session: {start_session}")
except FileNotFoundError:
    X, Dir, Train_Dir = [], [], []
    start_session = 0  # Start from the beginning if no previous data


try:
    for i, train_ses in enumerate(tqdm(training_sessions[start_session:]), start=start_session):
        print(f"Current session: {train_ses, i}")
        absd_dirs = [
            p.resolve() for p in Path(train_ses).glob("**") if p.is_dir() and ("ABSD" in p.name or "absd" in p.name)
        ]

        for absd_dir in absd_dirs:
            dt5_files = sorted(list(Path(absd_dir).glob("*.dt5")))

            if len(dt5_files) == 0:
                continue

            n_files = len(dt5_files)
            counter = 1
            skip_this = False
            for file in dt5_files:
                if not int(file.stem[-2:]) == n_files:
                    skip_this = True
                    continue
                if not counter == int(file.stem[-6:-4]):
                    skip_this = True
                    continue
                counter += 1

            if skip_this:
                continue

            # Process signals
            signals, ch_names = load_absd(absd_dir, return_is_updating=True, return_states=True)
            update_index = int(np.argwhere(np.array(ch_names) == 'is_updating'))
            update = np.argwhere(signals[update_index, :] == 1)
            print(f"Signals shape: {signals.shape}")

            new_sig = np.squeeze(signals[:, update])
            print(f"New signal shape: {new_sig.shape}")
            X.append(new_sig)
            Dir.append(ch_names)
            Train_Dir.append(absd_dir)



        # Save after every checkpoint interval

        np_array_X = np.empty(len(X), dtype=object)
        for j, x in enumerate(X):
            np_array_X[j] = x
        print(f"length of data: {len(X)}")

        np.save('Ch_train.npy', np.array(Dir, dtype=object), allow_pickle=True)
        np.save('Train_data.npy', np.array(np_array_X, dtype=object), allow_pickle=True)
        np.save('dir_train.npy', np.array(Train_Dir, dtype=object), allow_pickle=True)
        write_checkpoint(i)
        print(f"Checkpoint saved after session {i}")

except KeyboardInterrupt:
    print(f"Script interrupted at session {i}. Saving current progress...")
    print("Progress saved after interruption.")

except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Saving progress at session {i} before exiting...")
    raise  # Re-raise the exception after saving

else:
    # Final save after the loop completes successfully
    np_array_X = np.empty(len(X), dtype=object)
    for j, x in enumerate(X):
        np_array_X[j] = x
    print(f"length of data_: {len(X)}")

    np.save('Ch_train.npy', np.array(Dir, dtype=object), allow_pickle=True)
    np.save('Train_data.npy', np.array(np_array_X, dtype=object), allow_pickle=True)
    np.save('dir_train.npy', np.array(Train_Dir, dtype=object), allow_pickle=True)
    write_checkpoint(i + 1)
    print("Final save complete.")


  0%|          | 0/65 [00:00<?, ?it/s]

Current session: (PosixPath('/Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4'), 0)


2024-10-18 00:08:49.722 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_01of02.dt5
2024-10-18 00:09:27.639 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-10-18 00:09:27.646 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-10-18 00:09:32.267 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_13_59_55 (rec1)/ABSD data/2023_10_24_14_07_01_02of02.dt5
/var/folders/bv/m44_t4vj2sl8r_5ljvsxts2w0000gn/T/ipykernel_56503/2466790142.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from

Signals shape: (56, 353115)
New signal shape: (56, 62540)


2024-10-18 00:10:14.948 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_14_17_20 (rec2_HandOpen_Close)/ABSD data/2023_10_24_14_18_47_01of09.dt5
2024-10-18 00:11:06.128 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-10-18 00:11:06.129 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-10-18 00:11:10.835 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_14_17_20 (rec2_HandOpen_Close)/ABSD data/2023_10_24_14_18_47_02of09.dt5
2024-10-18 00:11:51.757 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_24_BSITraining_day4/WISCI/2023_10_24_14_17_20 (rec2_Ha

...

2024-10-18 15:37:04.000 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_23_BSITraining_day3_PhysioUpperLead/WIMAGINE/2023_10_23_14_18_43 (elbow extension)/ABSD data/2023_10_23_14_19_36_01of17.dt5
2024-10-18 15:37:28.310 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:196 - Scenario Name: classBCIViewerAvatarAllDegreesCourtine
2024-10-18 15:37:28.314 | INFO     | lighthouse.data_loader.load_chuv:load_dt5:199 - Sub Scenario Name: classBSISubScenarioUpperJointsClassif
2024-10-18 15:37:32.241 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_23_BSITraining_day3_PhysioUpperLead/WIMAGINE/2023_10_23_14_18_43 (elbow extension)/ABSD data/2023_10_23_14_19_36_02of17.dt5
2024-10-18 15:38:00.982 | INFO     | lighthouse.data_loader.load_chuv:concatenate_dt5:497 - Loading /Volumes/__UP2/0_RAW_DATA/UP2_001/UP2001_2023_10_23_BSITraining_day3_Phys

Signals shape: (56, 2955487)
New signal shape: (56, 395772)
length of data: 192


100%|██████████| 65/65 [15:36:54<00:00, 864.83s/it]

Checkpoint saved after session 64
length of data_: 192


Final save complete.
